In [2]:
import os
import glob
import pickle
from tqdm import tqdm # Per una bella barra di avanzamento
import numpy as np

from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

# 1. Caricare il modello InceptionV3 pre-addestrato su ImageNet
print("Caricamento del modello InceptionV3...")
base_model = InceptionV3(weights='imagenet')

Caricamento del modello InceptionV3...


In [3]:
# 2. Creare un nuovo modello che prende l'input di InceptionV3 e
#    restituisce l'output del layer 'avg_pool'.
#    Questo layer produce un vettore di 2048 feature per ogni immagine.
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
print("Modello per l'estrazione delle feature creato.")

Modello per l'estrazione delle feature creato.


In [4]:
# 3. Definire una funzione per processare una singola immagine
def extract_feature(img_path, model):
    """
    Carica un'immagine, la preprocessa per InceptionV3 e ne estrae le feature.
    """
    try:
        # InceptionV3 richiede immagini di dimensione 299x299
        img = image.load_img(img_path, target_size=(299, 299))
        
        # Converte l'immagine in un array NumPy
        x = image.img_to_array(img)
        
        # Aggiunge una dimensione extra per il "batch" (richiesto dal modello)
        x = np.expand_dims(x, axis=0)
        
        # Preprocessa l'input nello stesso modo in cui InceptionV3 è stato addestrato
        x = preprocess_input(x)
        
        # Esegue la predizione (estrazione feature)
        feature = model.predict(x, verbose=0)
        
        # L'output di avg_pool è (1, 1, 2048). Lo rimodelliamo in un vettore semplice (2048,).
        return feature.flatten() # .flatten() o .reshape(2048,) sono equivalenti qui

    except Exception as e:
        print(f"\nErrore durante l'elaborazione di {img_path}: {e}")
        return None


In [ ]:
# 4. Specificare il percorso delle immagini
images_path = r"..\data\Flickr8k_Dataset\Images" # Usa il percorso corretto
if not os.path.exists(images_path):
    raise FileNotFoundError(f"La cartella delle immagini non è stata trovata in: {images_path}")

In [12]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np
import os
import glob
import tqdm
import pickle

# Modello CNN pre-addestrato (InceptionV3)
base_model = InceptionV3(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

# Funzione per estrarre le feature da una singola immagine
def extract_feature(img_path, model):
    img = image.load_img(img_path, target_size=(299, 299))  # Inception richiede questa dimensione
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x, verbose=0)
    return feature.reshape(2048,)

# Percorso delle immagini
images_path = "../data/Flickr8k_Dataset/Images"
image_files = glob.glob(os.path.join(images_path, "*.jpg"))

# Crea cartella per salvare le feature
os.makedirs("features", exist_ok=True)

# Estrazione delle feature per tutte le immagini
features = {}
for img_path in tqdm.tqdm(image_files):
    image_id = os.path.basename(img_path).split('.')[0]
    features[image_id] = extract_feature(img_path, model)

# Salvataggio su file
with open("features/image_features.pkl", "wb") as f:
    pickle.dump(features, f)

print("✔️ Estrazione delle feature completata.")


96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 31s 0us/step


100%|██████████| 8091/8091 [27:58<00:00,  4.82it/s]  


✔️ Estrazione delle feature completata.
